In [7]:
from sagemaker import tuner
import sagemaker

session = sagemaker.Session()

In [2]:
%store -r tuning_job_name

In [3]:
%store -r s3_bucket_name

In [8]:
hyperparameter_tuner = tuner.HyperparameterTuner.attach(
    tuning_job_name, 
    sagemaker_session=session)

In [11]:
%%time

predictor = hyperparameter_tuner.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.large',
    endpoint_name='tuner-best-model-endpoint'
)


2021-05-21 14:21:28 Starting - Preparing the instances for training
2021-05-21 14:21:28 Downloading - Downloading input data
2021-05-21 14:21:28 Training - Training image download completed. Training in progress.
2021-05-21 14:21:28 Uploading - Uploading generated training model
2021-05-21 14:21:28 Completed - Training job completed
-------------!CPU times: user 212 ms, sys: 16.3 ms, total: 228 ms
Wall time: 6min 32s


In [12]:
!mkdir -p tmp

In [13]:
!aws s3 cp s3://{s3_bucket_name}/chapter06/input/test_data_no_header.csv tmp/test_data_no_header.csv

download: s3://sagemaker-cookbook-bucket/chapter06/input/test_data_no_header.csv to tmp/test_data_no_header.csv


In [14]:
import pandas as pd

test_data = pd.read_csv("tmp/test_data_no_header.csv", header=None)
test_data

,0,1,2,3,4
0,0,-13.956043,3.851976,-88,47
1,0,-4.690871,8.672612,22,76
2,1,9.064805,2.775964,-55,-74
3,1,5.459070,3.609733,58,71
4,0,-7.777093,4.053004,-72,-11
...,...,...,...,...,...
995,0,3.667547,14.248178,-85,70
996,1,11.253117,4.465519,54,-48
997,0,0.121657,9.131986,-19,23
998,0,2.191145,7.505053,20,14


In [15]:
test_data_no_target = test_data.drop(test_data.columns[0], axis=1)
test_data_no_target

,1,2,3,4
0,-13.956043,3.851976,-88,47
1,-4.690871,8.672612,22,76
2,9.064805,2.775964,-55,-74
3,5.459070,3.609733,58,71
4,-7.777093,4.053004,-72,-11
...,...,...,...,...
995,3.667547,14.248178,-85,70
996,11.253117,4.465519,54,-48
997,0.121657,9.131986,-19,23
998,2.191145,7.505053,20,14


In [16]:
csv_input = test_data_no_target.iloc[0:1].to_csv(index=False, header=False)

In [17]:
csv_input

'-13.956042970317096,3.8519760635213096,-88,47\n'

In [18]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(csv_input)

0.10131368786096573

In [19]:
predictor.delete_endpoint()